In [2]:
!pip install catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
import pickle
warnings.filterwarnings('ignore')


  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/35/7e/35fa1a7cf6925ff438e849cca50c88b8d28e02d9c3486442f2f85b86184a/catboost-1.2.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB 217.9 kB/s eta 0:07:45
   ---------------------------------------- 0.0/101.1 MB 259.2 kB/s eta 0:06:30
   ---------------------------------------- 0.0/101.1 MB 245.8 kB/s eta 0:06:52
   ---------------------------------------- 0.1/101.1 MB 599.1 kB/s eta 0:02:49
   ---------------------------------------- 0.3/101.1 MB 1.1 

In [10]:
df = pd.read_csv(r'E:\audi.csv')
display(df.head())


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [11]:
print(len(df))
display(df.shape)
display(df.dtypes)
display(df.isna().sum())
print(df.info())
display(df.describe())


10668


(10668, 9)

model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax               int64
mpg             float64
engineSize      float64
dtype: object

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB
None


,year,price,mileage,tax,mpg,engineSize
count,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000,10668.000000
mean,2017.100675,22896.685039,24827.244001,126.011436,50.770022,1.930709
std,2.167494,11714.841888,23505.257205,67.170294,12.949782,0.602957
min,1997.000000,1490.000000,1.000000,0.000000,18.900000,0.000000
25%,2016.000000,15130.750000,5968.750000,125.000000,40.900000,1.500000
50%,2017.000000,20200.000000,19000.000000,145.000000,49.600000,2.000000
75%,2019.000000,27990.000000,36464.500000,145.000000,58.900000,2.000000
max,2020.000000,145000.000000,323000.000000,580.000000,188.300000,6.300000


In [12]:
# Separate features and target variable
X = df.drop('price', axis=1).values
y = df['price'].values

# Label Encoding for 'model' and 'fuelType'
le_model = LabelEncoder()
le_fuelType = LabelEncoder()
X[:, 0] = le_model.fit_transform(X[:, 0])
X[:, -4] = le_fuelType.fit_transform(X[:, -4])

# One Hot Encoding for 'transmission'
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X = ct.fit_transform(X)

# Feature Scaling
sc = StandardScaler()
X = sc.fit_transform(X)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [18]:
rf_model = RandomForestRegressor(random_state=0)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print('Random Forest R2 Score:', r2_score(y_test, y_pred_rf))
print('Random Forest MAE:', mean_absolute_error(y_test, y_pred_rf))


Random Forest R2 Score: 0.9536134841307546
Random Forest MAE: 1538.730980670462


In [19]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print('Linear Regression R2 Score:', r2_score(y_test, y_pred_lr))
print('Linear Regression MAE:', mean_absolute_error(y_test, y_pred_lr))


Linear Regression R2 Score: 0.7916670925031881
Linear Regression MAE: 3381.9167483000706


In [20]:
et_model = ExtraTreesRegressor(n_estimators=120)
et_model.fit(X_train, y_train)
y_pred_et = et_model.predict(X_test)
print('Extra Trees R2 Score:', r2_score(y_test, y_pred_et))
print('Extra Trees MAE:', mean_absolute_error(y_test, y_pred_et))


Extra Trees R2 Score: 0.9570879357066128
Extra Trees MAE: 1529.0914597000938


In [21]:
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=80, stop=1500, num=10)],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(6, 45, num=5)],
    'min_samples_split': [2, 5, 10, 15, 100],
    'min_samples_leaf': [1, 2, 5, 10]
}
rf_random = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, scoring='neg_mean_squared_error', n_iter=3, cv=3, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)
best_rf_model = rf_random.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)
print('Tuned Random Forest R2 Score:', r2_score(y_test, y_pred_best_rf))
print('Tuned Random Forest MAE:', mean_absolute_error(y_test, y_pred_best_rf))


Tuned Random Forest R2 Score: 0.9589123068559103
Tuned Random Forest MAE: 1504.967469293171


In [16]:
cat_model = CatBoostRegressor()
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
print('CatBoost R2 Score:', r2_score(y_test, y_pred_cat))
print('CatBoost MAE:', mean_absolute_error(y_test, y_pred_cat))


Learning rate set to 0.057452
0:	learn: 11184.0694779	total: 161ms	remaining: 2m 40s
1:	learn: 10690.2138498	total: 167ms	remaining: 1m 23s
2:	learn: 10221.1011877	total: 172ms	remaining: 57s
3:	learn: 9778.8067263	total: 176ms	remaining: 43.9s
4:	learn: 9382.6454054	total: 181ms	remaining: 36s
5:	learn: 8989.9290587	total: 187ms	remaining: 30.9s
6:	learn: 8628.4892653	total: 192ms	remaining: 27.3s
7:	learn: 8280.9179062	total: 197ms	remaining: 24.5s
8:	learn: 7950.5476053	total: 202ms	remaining: 22.2s
9:	learn: 7636.7735365	total: 208ms	remaining: 20.6s
10:	learn: 7349.6084699	total: 212ms	remaining: 19.1s
11:	learn: 7084.0457263	total: 217ms	remaining: 17.8s
12:	learn: 6842.7060927	total: 223ms	remaining: 16.9s
13:	learn: 6608.0244743	total: 227ms	remaining: 16s
14:	learn: 6387.7285726	total: 231ms	remaining: 15.2s
15:	learn: 6170.1354894	total: 237ms	remaining: 14.6s
16:	learn: 5973.9133066	total: 242ms	remaining: 14s
17:	learn: 5780.4713044	total: 246ms	remaining: 13.4s
18:	learn: 

156:	learn: 2458.9977863	total: 946ms	remaining: 5.08s
157:	learn: 2454.9587035	total: 950ms	remaining: 5.07s
158:	learn: 2451.9873804	total: 956ms	remaining: 5.05s
159:	learn: 2450.8561954	total: 961ms	remaining: 5.04s
160:	learn: 2449.7476635	total: 965ms	remaining: 5.03s
161:	learn: 2446.4227836	total: 971ms	remaining: 5.02s
162:	learn: 2444.2555123	total: 976ms	remaining: 5.01s
163:	learn: 2441.5688551	total: 981ms	remaining: 5s
164:	learn: 2440.2986221	total: 987ms	remaining: 4.99s
165:	learn: 2437.3232663	total: 992ms	remaining: 4.98s
166:	learn: 2432.4649721	total: 996ms	remaining: 4.97s
167:	learn: 2428.3970878	total: 1s	remaining: 4.96s
168:	learn: 2423.6253903	total: 1.01s	remaining: 4.95s
169:	learn: 2421.4159298	total: 1.01s	remaining: 4.93s
170:	learn: 2420.5105888	total: 1.01s	remaining: 4.92s
171:	learn: 2419.1046943	total: 1.02s	remaining: 4.91s
172:	learn: 2415.9697367	total: 1.02s	remaining: 4.9s
173:	learn: 2414.6048787	total: 1.03s	remaining: 4.88s
174:	learn: 2413.

325:	learn: 2158.9245109	total: 1.71s	remaining: 3.54s
326:	learn: 2157.0096074	total: 1.72s	remaining: 3.53s
327:	learn: 2156.0953247	total: 1.72s	remaining: 3.52s
328:	learn: 2155.2997990	total: 1.72s	remaining: 3.51s
329:	learn: 2154.0676384	total: 1.73s	remaining: 3.51s
330:	learn: 2152.1472733	total: 1.73s	remaining: 3.5s
331:	learn: 2151.6773078	total: 1.73s	remaining: 3.49s
332:	learn: 2151.1380063	total: 1.74s	remaining: 3.48s
333:	learn: 2149.9980622	total: 1.74s	remaining: 3.47s
334:	learn: 2148.6561715	total: 1.75s	remaining: 3.46s
335:	learn: 2147.5979814	total: 1.75s	remaining: 3.46s
336:	learn: 2146.4366203	total: 1.75s	remaining: 3.45s
337:	learn: 2145.3221621	total: 1.76s	remaining: 3.44s
338:	learn: 2144.1606578	total: 1.76s	remaining: 3.43s
339:	learn: 2143.4114760	total: 1.76s	remaining: 3.42s
340:	learn: 2141.9134923	total: 1.77s	remaining: 3.41s
341:	learn: 2141.5027453	total: 1.77s	remaining: 3.41s
342:	learn: 2140.5973868	total: 1.77s	remaining: 3.4s
343:	learn: 

478:	learn: 2024.4385398	total: 2.29s	remaining: 2.49s
479:	learn: 2023.7573534	total: 2.29s	remaining: 2.48s
480:	learn: 2022.9267016	total: 2.3s	remaining: 2.48s
481:	learn: 2022.5285079	total: 2.3s	remaining: 2.47s
482:	learn: 2021.6201313	total: 2.31s	remaining: 2.47s
483:	learn: 2020.6956057	total: 2.31s	remaining: 2.46s
484:	learn: 2019.6415235	total: 2.31s	remaining: 2.46s
485:	learn: 2019.2660694	total: 2.32s	remaining: 2.45s
486:	learn: 2018.9854237	total: 2.32s	remaining: 2.44s
487:	learn: 2017.8809698	total: 2.33s	remaining: 2.44s
488:	learn: 2016.5544473	total: 2.33s	remaining: 2.43s
489:	learn: 2016.2635297	total: 2.33s	remaining: 2.43s
490:	learn: 2015.9216242	total: 2.34s	remaining: 2.42s
491:	learn: 2015.3919735	total: 2.34s	remaining: 2.42s
492:	learn: 2014.7855567	total: 2.34s	remaining: 2.41s
493:	learn: 2014.4619330	total: 2.35s	remaining: 2.4s
494:	learn: 2013.9176550	total: 2.35s	remaining: 2.4s
495:	learn: 2013.6272291	total: 2.35s	remaining: 2.39s
496:	learn: 20

632:	learn: 1928.2221433	total: 2.88s	remaining: 1.67s
633:	learn: 1927.9877195	total: 2.88s	remaining: 1.66s
634:	learn: 1927.2350665	total: 2.88s	remaining: 1.66s
635:	learn: 1927.0566618	total: 2.89s	remaining: 1.65s
636:	learn: 1926.4536636	total: 2.89s	remaining: 1.65s
637:	learn: 1925.9713985	total: 2.89s	remaining: 1.64s
638:	learn: 1925.0475491	total: 2.9s	remaining: 1.64s
639:	learn: 1924.7392229	total: 2.9s	remaining: 1.63s
640:	learn: 1924.2557266	total: 2.91s	remaining: 1.63s
641:	learn: 1923.6991232	total: 2.91s	remaining: 1.62s
642:	learn: 1923.1585026	total: 2.91s	remaining: 1.62s
643:	learn: 1922.2830460	total: 2.92s	remaining: 1.61s
644:	learn: 1921.6582178	total: 2.92s	remaining: 1.61s
645:	learn: 1921.4140145	total: 2.92s	remaining: 1.6s
646:	learn: 1920.7957001	total: 2.93s	remaining: 1.6s
647:	learn: 1920.4441658	total: 2.93s	remaining: 1.59s
648:	learn: 1919.3724824	total: 2.94s	remaining: 1.59s
649:	learn: 1919.1538698	total: 2.94s	remaining: 1.58s
650:	learn: 19

830:	learn: 1834.5549875	total: 3.62s	remaining: 737ms
831:	learn: 1834.0988549	total: 3.63s	remaining: 733ms
832:	learn: 1833.8189952	total: 3.63s	remaining: 728ms
833:	learn: 1833.3335242	total: 3.63s	remaining: 724ms
834:	learn: 1832.9733766	total: 3.64s	remaining: 719ms
835:	learn: 1832.8071890	total: 3.64s	remaining: 715ms
836:	learn: 1832.5660391	total: 3.65s	remaining: 710ms
837:	learn: 1831.9180126	total: 3.65s	remaining: 706ms
838:	learn: 1831.1675342	total: 3.66s	remaining: 702ms
839:	learn: 1830.7563210	total: 3.66s	remaining: 697ms
840:	learn: 1830.3246842	total: 3.66s	remaining: 693ms
841:	learn: 1830.0657664	total: 3.67s	remaining: 688ms
842:	learn: 1829.4229004	total: 3.67s	remaining: 684ms
843:	learn: 1828.7315034	total: 3.67s	remaining: 679ms
844:	learn: 1828.4512671	total: 3.68s	remaining: 675ms
845:	learn: 1827.8254092	total: 3.68s	remaining: 670ms
846:	learn: 1827.2232798	total: 3.69s	remaining: 666ms
847:	learn: 1826.6298672	total: 3.69s	remaining: 661ms
848:	learn

987:	learn: 1774.5222134	total: 4.21s	remaining: 51.1ms
988:	learn: 1774.2816872	total: 4.21s	remaining: 46.8ms
989:	learn: 1773.8650257	total: 4.21s	remaining: 42.6ms
990:	learn: 1773.1775081	total: 4.22s	remaining: 38.3ms
991:	learn: 1772.8075712	total: 4.22s	remaining: 34ms
992:	learn: 1772.4544536	total: 4.22s	remaining: 29.8ms
993:	learn: 1772.2659066	total: 4.23s	remaining: 25.5ms
994:	learn: 1772.0462978	total: 4.23s	remaining: 21.3ms
995:	learn: 1771.2044114	total: 4.24s	remaining: 17ms
996:	learn: 1770.9048234	total: 4.24s	remaining: 12.8ms
997:	learn: 1770.4500877	total: 4.24s	remaining: 8.51ms
998:	learn: 1770.1346179	total: 4.25s	remaining: 4.25ms
999:	learn: 1769.8343384	total: 4.25s	remaining: 0us
CatBoost R2 Score: 0.9641612028134969
CatBoost MAE: 1453.5230700749125


In [22]:
with open('model.pkl', 'wb') as f:
    pickle.dump(best_rf_model, f)


In [23]:
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
predictions = loaded_model.predict(X_test)
print(predictions)


[14004.68894762 23755.96128649 28531.69820609 ... 48126.78996325
 31336.31273356 10083.11437582]


In [24]:
predictions = loaded_model.predict(X_test)
actual_prices = y_test

# Create a DataFrame to compare predictions with actual prices
comparison = pd.DataFrame({'Actual': actual_prices, 'Predicted': predictions})
display(comparison.head())


,Actual,Predicted
0,14998,14004.688948
1,21950,23755.961286
2,28990,28531.698206
3,25489,27451.508565
4,30950,31881.532685
